In [3]:
import pandas as pd
import h3
import random


In [4]:
# Define the region bounds for South Holland, Netherlands (approximately)
south_holland_bounds = {
    "min_lat": 51.57,
    "max_lat": 52.24,
    "min_lon": 3.36,
    "max_lon": 5.13
}

In [5]:
# Function to generate a random location within the specified bounds
def generate_random_location(bounds):
    lat = random.uniform(bounds["min_lat"], bounds["max_lat"])
    lon = random.uniform(bounds["min_lon"], bounds["max_lon"])
    return lat, lon

In [7]:
# Generate mock data for H3 hexagons
num_points = 1000
resolution = 9  # H3 resolution level

In [8]:
data = []
for _ in range(num_points):
    lat, lon = generate_random_location(south_holland_bounds)
    hex_address = h3.geo_to_h3(lat, lon, resolution)
    random_value = random.randint(1, 1000)  # Random value between 1 and 1000
    data.append({
        "hex_address": hex_address,
        "latitude": lat,
        "longitude": lon,
        "value": random_value
    })

# Create a DataFrame
df = pd.DataFrame(data)

In [13]:
# Save to CSV
csv_path = "./hex/h3_farm_mock_data.csv"
df.to_csv(csv_path, index=False)

OSError: Cannot save file into a non-existent directory: 'hex'